# Bricklayer Robot

In [2]:
# Extensions: magic to avoid re-loading modified .py modules all the time
%load_ext autoreload
%autoreload 2

# Bookkeeping
import sys
import shutil
import os as os
import matplotlib.pyplot as plt

# Custom project classes and functions
from iiwa import *
from Trajectory import *
from TrajectoryBuilder import *
from Visualizer import *
from BrickPlanner import *
from HighPerformanceTrajectoryBuilder import *

# Some paths
kProjectDir = os.getcwd()
print("Working in directory ", kProjectDir)

# Hack: move our model packages to the manipulation/ folder to allow Drake to find them
shutil.copyfile(kProjectDir + "/models/brick.dmd.yaml", kProjectDir + "/../manipulation/manipulation/models/brick.dmd.yaml")
shutil.copyfile(kProjectDir + "/models/real_brick.sdf", kProjectDir + "/../manipulation/manipulation/models/real_brick.sdf")
shutil.copyfile(kProjectDir + "/models/ground_model.sdf", kProjectDir + "/../manipulation/manipulation/models/ground_model.sdf")

# Start the visualizer.
meshcat = StartMeshcat()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/nikita/env/lib/python3.10/site-packages/mpld3/mplexporter/renderers/base.py:4: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
INFO:drake:Meshcat listening for connections at http://localhost:7000


Working in directory  /home/nikita/MIT_6_4212/project


In [13]:
# System configs
# Brick configuration
kNumberOfBricks =  50 # @param
# Brick dimensions, must be consistent with the geometry in models/real_brick.sdf
kBrickGeom = np.array([0.075, 0.1, 0.05]) # @param

# Brick warehouse config
kWhLocation = np.array([2.5, 2.5, 0.15]) # location of the warehouse (corner), # @param
kWhSize = np.array([6, 6, 10]) # size of the warehouse grid in cells, # @param
if (kWhSize.prod() < kNumberOfBricks):
    assert False, "The warehouse is too small to fit all bricks"

In [14]:
# Generate bricks
floorplan = """
|
|
|
"""

floorplan_0 = """
|- - - - - |
|          |
|          |
|          |
|- - - - - |
"""

floorplan_1 = """
|- - - - - - - - |
|                |
|                |
|                |
|                |
|                |
|                |
|                |
|                |
|- - - - - - - - |
"""

floorplan_2 = """
|- - - - - - - - |
|                |
|                |
|                |
|                |- - - |
|                |      |
|                |      |
|                |- - - |
|                |
|- - - - - - - - |
"""

kHeight = 1 # @param
kGap = 0.005 # @param
planer = Planner(floorplan_2, kBrickGeom, kGap, kHeight)
planer.shift(-0.55, -0.45)
total_bricks_needed = planer.get_total_bricks_n()
if (total_bricks_needed > kNumberOfBricks):
    assert False, "There is not enough bricks to cover the building"
print("Bricks planned, total bricked needed: ", total_bricks_needed)

# Visualize floorplan
vis = Visualizer(meshcat, kBrickGeom)
vis.clear_vis()
vis.visualize_plan(np.array(planer.get_brick_poses()))

Parsing floorplan:  
|- - - - - - - - |
|                |
|                |
|                |
|                |- - - |
|                |      |
|                |      |
|                |- - - |
|                |
|- - - - - - - - |

Bricks planned, total bricked needed:  46


In [39]:
# Robot setup
kNumRobots = 1
kArms = [
         np.array([0, 0, 0]),
         np.array([-0.5, 1.0, 0])
        ]
kBrickSources = [
                 RigidTransform(np.array([0.6, 0.0, kBrickGeom[2]/2])),
                 RigidTransform(np.array([0.0, 1.0, kBrickGeom[2]/2]))
                ]

ilyich = IIWA_Ilyich(meshcat, kNumberOfBricks, kBrickGeom, kArms, None)

In [40]:
# Build trajectories for robots
def build_trajectories_for_robot(robot_id, bricks):
    X_WBrickSource = kBrickSources[robot_id]
    robot_pose = RigidTransform(RollPitchYaw(0, 0, 0), kArms[robot_id])
    X_WG = ilyich.get_X_WG()[robot_id]

    # Build initial trajectory and get q0 in the calibration point
    # np.array([ 0., 0.6, 0., 0.0, 0.0, 1, 0.0, 0., 0.])
    trj_builder = TrajectoryBuilder(X_WBrickSource)
    trj_builder.gen_initial_traj(X_WG)
    res, q0 = trj_builder.solve_IK(np.array([ 0., 0.0, 0., 0.0, 0.0, 0.0, 0.0, 0., 0.]), robot_pose)
    assert res == True, "Failed to solve IK for the initial trajectory"
    print("q0 in calibration point is ", q0)

    # Build other trajectories
    kNumWorkers = 20 # @Param
    hpc_trj_builder = HighPerformanceTrajectoryBuilder(planer,
                                                       X_WBrickSource,
                                                       bricks,
                                                       kNumWorkers,
                                                       q0,
                                                       robot_pose)
    covered_bricks, failed_bricks = hpc_trj_builder.solve(trj_builder)
    print("Covered bricks: ", covered_bricks)
    print("Failed bricks:", failed_bricks)

    # Speed-up a bit
    trj_builder.get_trajectories().slow_down(0.5)

    # Make trajectories for iiwa with IK
    return trj_builder.get_trajectories().form_iiwa_traj_q(), covered_bricks, failed_bricks

# Visualize coverage
def visualize_coverage(covered_bricks, failed_bricks, covered_color, clear=True):
    # Visualize coverage
    vis = Visualizer(meshcat, kBrickGeom)
    if clear:
        vis.clear_vis()
    vis.visualize_coverage(planer.get_brick_poses()[covered_bricks],
                           planer.get_brick_poses()[failed_bricks],
                           covered_color)

    # Visualize all trajectories
    for b_id in covered_bricks:
        points = trj_builder.get_trajectory_points_for_brick(b_id)
        vis.visualize_traj(points, b_id)

# Do 1st robot
final_trajectories = []

(traj, finger_traj), covered_bricks, failed_bricks = build_trajectories_for_robot(0, range(0, total_bricks_needed))
visualize_coverage(covered_bricks, failed_bricks, Rgba(0.61, 1, 0.60), clear=True)
final_trajectories.append((traj, finger_traj))

# Do 2nd robot
(traj_1, finger_traj_1), covered_bricks_1, failed_bricks_1 = build_trajectories_for_robot(1, failed_bricks)
visualize_coverage(covered_bricks_1, failed_bricks_1, Rgba(1, 1, 0.60), clear=False)
final_trajectories.append((traj_1, finger_traj_1))

q0 in calibration point is  [ 0.14557608  0.67413625 -0.22359343 -1.39006593  0.15704599  1.09444912
  1.46900663  0.          0.        ]
Initializing HPC trajectory builder with  20  workers on  192  core machine
Covered bricks:  [0, 1, 2, 3, 4, 5, 6, 7, 10, 12, 13, 14, 15, 16, 17, 18, 19, 22, 23, 25, 26, 28, 29, 30, 31, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
Failed bricks: [8, 9, 11, 20, 21, 24, 27, 32, 33]
q0 in calibration point is  [ 0.15244708  0.45644639 -0.19379517 -1.7365921   0.10379735  0.95790411
  1.48881621  0.          0.        ]
Initializing HPC trajectory builder with  20  workers on  192  core machine
Covered bricks:  [8, 9, 11, 20, 21, 24, 27, 32, 33]
Failed bricks: []


In [34]:
# Create robot
ilyich = IIWA_Ilyich(meshcat, kNumberOfBricks, kBrickGeom, kArms, final_trajectories)

# Create default context
context = ilyich.CreateDefaultContext()

# Put all bricks in the warehouse
ilyich.put_bricks_in_warehouse(context, kWhLocation, kWhSize, kNumberOfBricks)

/home/nikita/MIT_6_4212/project/iiwa.py:163: DrakeDeprecationWarning: (Deprecated) See ForcedPublish() 
*/ (Deprecated.)

Deprecated:
    Use ForcedPublish() instead This will be removed from Drake on or
    after 2023-03-01.
  self.diagram.Publish(context)


In [35]:
#
# Move
#
# Move to calibration point and calibrate
sim, ctx = ilyich.work(None, context, sim_duration=trj_builder.get_trajectories().get_breakpoints()[0])

# Continue for all bricks
for i, b_id in enumerate(covered_bricks):
    # Show trajectory
    points = trj_builder.get_trajectory_points_for_brick(b_id)
    vis.clear_traj()
    vis.visualize_traj(points)

    # Teleport the next new brick from the warehouse and unlock it
    ilyich.move_brick(context, X_WBrickSource, i)
    ilyich.unlock_brick(context, i)

    # Manipulate the brick to target
    sim, ctx = ilyich.work(sim, ctx, sim_duration=trj_builder.get_trajectories().get_breakpoints()[i+1])

    # Lock brick
    ilyich.lock_brick(context, i)

KeyboardInterrupt: 